<a href="https://colab.research.google.com/github/yaoshiang/churn_predictor/blob/master/Churn_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Churn predictor

## This tool uses 5 months of churn to predict the churn and subscriber totals for the full 120 months. 

(c) 2020 Yaoshiang Ho



```
# This is formatted as code
```

Enter your six months of end-of-month subscriber numbers here, including month 0 (all sub adds prior to end of month). 

As a sample, we used 1000, 600, 420, 315, 252, and 215. This represents churn of 40%, 30%, 25%, 20%, and 15%. 



In [0]:
subs=[1.000, .600, .420, .315, .252, .215]

This is the churn rate per month:

In [45]:
import numpy as np

subs = np.array(subs)

churn = (subs[0:5] - subs[1:6]) / subs[0:5]
print(churn)

[0.4       0.3       0.25      0.2       0.1468254]


Setup 99 segments with churn ranging from 1% to 99%


In [46]:
segment_churns = np.arange(.01,1., 0.01)
print(segment_churns)

[0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09 0.1  0.11 0.12 0.13 0.14
 0.15 0.16 0.17 0.18 0.19 0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28
 0.29 0.3  0.31 0.32 0.33 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42
 0.43 0.44 0.45 0.46 0.47 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56
 0.57 0.58 0.59 0.6  0.61 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7
 0.71 0.72 0.73 0.74 0.75 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84
 0.85 0.86 0.87 0.88 0.89 0.9  0.91 0.92 0.93 0.94 0.95 0.96 0.97 0.98
 0.99]


In [0]:
import keras.backend as K

months = 5

def fprop(segment_sizes):
  '''
  Arguments: 
  * segment_sizes: numpy array of size (99)
  * segment_churns: numpy array of size (99)
  Returns:
  * np array shape (months, total subs)
  '''
  retval = []
  # retval2 = []
  current_month_sizes = segment_sizes
  for i in range(5):
    current_month_sizes = current_month_sizes * (1. - segment_churns) # m,99 * 99 = m,99
    retval.append(K.sum(current_month_sizes, axis=-1)) # m
    # retval2.append(current_month_sizes)

  retval = K.stack(retval, axis=1) # m,99,6

  return retval

In [0]:
from keras.models import Model
from keras.layers import Lambda, Input, Dense
from keras.utils import to_categorical
from keras import optimizers
from keras import regularizers

kx = Input(shape=(1,))
kd = Dense(99, use_bias=False, activation='softmax', activity_regularizer=regularizers.l1_l2())(kx) 
k_y = Lambda(fprop, output_shape=(5,))(kd)
m = Model(inputs=kx, outputs=k_y)

In [49]:
opt=optimizers.Adam(lr=.1)

m.compile(optimizer=opt, loss='mse')

x = np.ones(shape=(1000,1))
y = np.tile(subs[1:], (1000,1))
m.fit(x=x, y=y, epochs=5, batch_size=1)

Epoch 1/5
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0101
Epoch 2/5
1000/1000 [==============================] - 1s 1ms/step - loss: 0.0101
Epoch 3/5
1000/1000 [==============================] - 1s 1ms/step - loss: 0.0101
Epoch 4/5
1000/1000 [==============================] - 1s 1ms/step - loss: 0.0101
Epoch 5/5
1000/1000 [==============================] - 1s 1ms/step - loss: 0.0101


In [50]:
r = m.predict(x=np.ones(shape=(1)))
print(r)

[[0.5990017  0.41804105 0.31760418 0.25470227 0.21197775]]


In [51]:
l = m.layers[1].get_weights()
# print(l[0])

sizes = K.eval(K.softmax(l[0]))
print(sizes)

[[0.01404917 0.01381174 0.01360285 0.01342117 0.0132645  0.01313122
  0.0130197  0.01292803 0.01285474 0.01279834 0.01275737 0.01273034
  0.01271583 0.01271263 0.01271929 0.01273476 0.01275766 0.01278694
  0.01282161 0.01286055 0.01290271 0.01294743 0.01299347 0.01303995
  0.01308599 0.01313114 0.01317473 0.0132158  0.01325354 0.01328761
  0.01331744 0.01334246 0.01336215 0.01337593 0.01338343 0.01338426
  0.01337844 0.01336499 0.01334392 0.01331484 0.01327772 0.01323234
  0.01317861 0.01311627 0.01304509 0.01296502 0.01287603 0.01277805
  0.01267114 0.01255537 0.01243075 0.01229739 0.01215535 0.01200473
  0.01184568 0.01167824 0.01150262 0.01131903 0.01112756 0.01092864
  0.01072238 0.01050904 0.01028898 0.01006254 0.00982984 0.0095914
  0.00934734 0.00909814 0.00884423 0.00858575 0.00832335 0.00805701
  0.00778744 0.00751473 0.00723947 0.00696201 0.00668214 0.00640149
  0.00612016 0.00583804 0.00555627 0.0052741  0.00499369 0.00471348
  0.00443956 0.00416245 0.00389329 0.0036217  0.0

In [0]:
def build(sizes, churn_segs, months=120):
  '''
  returns
  * size of cohort for each month
  * detail per churn segment
  * churns
  * estimated lifetime of each sub in months. 
  '''

  initial_subs = np.sum(sizes)

  summary = np.zeros(shape=(months+1))
  detail = np.zeros(shape=(months+1, churn_segs.shape[0]))
  churns = np.zeros(shape=(months+1))

  current = sizes
  detail[0] = current

  # Forward prop through the months 
  for i in range(1, months+1):
    current = current * (1. - churn_segs)
    detail[i] = current

  summary = np.sum(detail, axis=1)

  # Calculate churns 
  churns[0] = 0
  churns[1:] = (summary[:-1] - summary[1:]) / summary[:-1]

  lt = np.sum(summary) / initial_subs

  return summary, detail, churns, lt

summary, detail, churns, lt = build(sizes, segment_churns)

This is the chance that a sub is still around in any given month. 

In [53]:
print(summary)

[1.00000005 0.59900168 0.41804103 0.31760416 0.25470227 0.21197774
 0.18121103 0.15805817 0.14002837 0.12559989 0.11379461 0.10395699
 0.09563215 0.08849497 0.08230712 0.07689002 0.07210728 0.06785295
 0.06404348 0.06061212 0.05750488 0.05467764 0.05209396 0.04972349
 0.04754077 0.04552423 0.04365552 0.04191891 0.04030083 0.03878951
 0.03737472 0.03604748 0.03479991 0.03362506 0.03251675 0.0314695
 0.03047841 0.0295391  0.02864763 0.02780046 0.02699439 0.02622651
 0.0254942  0.02479506 0.0241269  0.02348774 0.02287575 0.02228925
 0.0217267  0.02118668 0.0206679  0.02016912 0.01968925 0.01922725
 0.01878214 0.01835305 0.01793913 0.01753962 0.01715379 0.01678096
 0.01642051 0.01607185 0.01573442 0.01540769 0.01509119 0.01478446
 0.01448706 0.01419858 0.01391865 0.0136469  0.01338299 0.0131266
 0.01287742 0.01263516 0.01239956 0.01217035 0.01194728 0.01173012
 0.01151865 0.01131266 0.01111195 0.01091632 0.0107256  0.01053961
 0.01035819 0.01018118 0.01000842 0.00983978 0.00967512 0.009514

In [54]:
print (detail)

[[1.40491715e-002 1.38117373e-002 1.36028500e-002 ... 1.41012354e-003
  1.20671710e-003 1.02623459e-003]
 [1.39086798e-002 1.35355026e-002 1.31947645e-002 ... 4.23037063e-005
  2.41343421e-005 1.02623459e-005]
 [1.37695930e-002 1.32647925e-002 1.27989216e-002 ... 1.26911119e-006
  4.82686842e-007 1.02623459e-007]
 ...
 [4.29144623e-003 1.27328388e-003 3.73847562e-004 ... 2.81556281e-183
  4.01000540e-204 1.02623459e-239]
 [4.24853177e-003 1.24781820e-003 3.62632136e-004 ... 8.44668844e-185
  8.02001079e-206 1.02623459e-241]
 [4.20604645e-003 1.22286184e-003 3.51753172e-004 ... 2.53400653e-186
  1.60400216e-207 1.02623459e-243]]


These are the churn rates per month. 

In [55]:
print(churns)

[0.         0.40099835 0.30210374 0.24025602 0.19805121 0.16774303
 0.14514124 0.12776735 0.11407071 0.10303965 0.09399119 0.08645063
 0.08007964 0.0746316  0.06992319 0.06581567 0.06220238 0.05900008
 0.05614302 0.05357859 0.05126425 0.04916526 0.04725299 0.04550362
 0.04389724 0.04241701 0.04104864 0.03977993 0.03860037 0.03750089
 0.0364736  0.03551163 0.03460895 0.03376025 0.03296083 0.03220653
 0.03149364 0.03081887 0.03017923 0.02957208 0.028995   0.02844583
 0.02792261 0.02742355 0.02694704 0.02649158 0.02605583 0.02563854
 0.02523859 0.02485492 0.02448657 0.02413267 0.02379237 0.02346494
 0.02314966 0.02284587 0.02255299 0.02227043 0.02199767 0.02173423
 0.02147965 0.02123348 0.02099535 0.02076486 0.02054166 0.02032543
 0.02011585 0.01991263 0.01971548 0.01952417 0.01933842 0.01915802
 0.01898274 0.01881238 0.01864674 0.01848564 0.01832889 0.01817633
 0.01802781 0.01788316 0.01774225 0.01760494 0.0174711  0.01734061
 0.01721335 0.0170892  0.01696807 0.01684984 0.01673442 0.0166

This is your expected months of lifetime per sub.

In [56]:
print(lt)

6.058713830107121


This is your average churn for the segment. 

In [57]:
print(1/lt)

0.16505153206457343
